In [77]:
import itertools as it
from multiprocessing import Pool, cpu_count
from collections import defaultdict

with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
"""


Ok. so. I think what's important is the data structure in this case I think it's
`dict{A:[(perimeters,set() for positions), (perimeters,set() for positions), ...]}` with `(perimeters,set() for positions)` for every area with the same symbol. How would I detect that?

```
4+4+4+4
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO

1*4+5*12 <- correct
6*16 <- incorrect
OOOOO
OXOXO
OOOXO
OXXXO
OOOOO
```

### Part 1

I was then a bit stuck on how to calculate a cluster. Flood fill, of course -- I've read a method using recursion but the queue method made more intuitive sense to me, so I ended up using that instead.

### Part 2

Stuck on this for a while because I first implemented just comparing the same axis, which means all different sides on the same axis are counted as 1. 

In [84]:
# ref: yossi_peti https://www.reddit.com/r/adventofcode/comments/1hcdnk0/comment/m1nfo1q/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
class GardenPlots():
    def __init__(self,txt):
        self.matrix = [[c for c in line]for line in txt.splitlines()]
        self.h,self.w = len(self.matrix), len(self.matrix[0])
        self.dirs = [(0,1),(1,0),(-1,0),(0,-1)]

    def get(self,tup):
        x,y=tup
        if 0<=x<self.h and 0<=y<self.w:
            return self.matrix[x][y]
        else: return None

    def calculate_score(self,bulk_discount = False):
        uncovered = set()
        for x in range(self.h):
            for y in range(self.w):
                uncovered.add((x,y))
        score = 0 
        while len(uncovered)>0:
            pos = uncovered.pop() # randomly find a starting point from remaining set
            # print(self.get(pos))
            cluster = self.flood_cluster(pos)
            if bulk_discount:
                score += self.price_by_sides(cluster) # part 2
            else:
                score += self.price(cluster) # part 1
            uncovered -= cluster
            # print("uncovered set", uncovered)
        return score

    def price(self,cluster):
        area,perimeter = len(cluster),0
        for pos in cluster:
            # print("     ",pos)
            perimeter += 4
            for dir in self.dirs: # 4 fence for every pos, check neighbors
                new_pos = (pos[0]+dir[0], pos[1]+dir[1])
                if new_pos in cluster: # if it's in the cluster, removed corresponding edges
                    perimeter -= 1
        # print(area,"*",perimeter)
        return area * perimeter
    
    def price_by_sides(self,cluster):
        area,sides = len(cluster),0
        perimeter = set()
        for pos in cluster:
            for dir in self.dirs: # 4 fence for every pos, check neighbors
                new_pos = (pos[0]+dir[0], pos[1]+dir[1])
                if new_pos not in cluster: # if it's in the cluster, removed corresponding edges
                    perimeter.add((new_pos,dir))

        while len(perimeter)>0:
            pos,dir = perimeter.pop()
            sides += 1
            # peris = perimeter.copy()
            getdelta = lambda t: (1,0) if t[0] == 0 else (0,1)
            d = getdelta(dir)
            # for one edge, go to its relative left
            r_next = (pos[0]+d[0],pos[1]+d[1])
            while(r_next,dir) in perimeter:
                perimeter.remove((r_next,dir))
                r_next = (r_next[0]+d[0],r_next[1]+d[1])
            l_next = (pos[0]-d[0],pos[1]-d[1])
            # then go right
            while(l_next,dir) in perimeter:
                perimeter.remove((l_next,dir))
                l_next = (l_next[0]-d[0],l_next[1]-d[1])

            # print("perimeter: ", perimeter)
        # print(area,"*",sides)
        return area * sides

    def flood_cluster(self, start): 
        # from start: 
        # until queue runs out: check all unseen nodes that are start
        # add it to set
        # return a set of coords been

        cluster = set()
        q = [start]
        c = self.get(start)

        while len(q) > 0:
            current_pos = q.pop()
            cluster.add(current_pos)
            for dir in self.dirs:
                new_pos = (current_pos[0]+dir[0], current_pos[1]+dir[1])
                if new_pos not in cluster and self.get(new_pos) == c:
                    q.append(new_pos)

        return cluster # a set of (x,y)        

res = GardenPlots(input)
# res.matrix
print("part1:",res.calculate_score())
print("part2:",res.calculate_score(True))


part1: 1344578
part2: 814302


Takeaways:
- This was one of the cases where if I'd used complex numbers it would've been way simpler. it's just more suitable for matrix/directional calculations (i.e. part 2 with the moving to relative left/right stuff...)
- for example if directions are represented as `d in [1, -1, 1j, -1j]`, a point's relative left is simply `pos + d*j`
```
d*j
1*j -> j  # x
-1*j -> -j 
j*j -> -1 # y
-j*j -> 1
```
